In [18]:
#importing libraries
import random
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import re
from collections import Counter
import nltk
from nltk.corpus import stopwords
from nltk import word_tokenize
from collections import Counter
from sklearn.linear_model import LinearRegression
from itertools import product
from sklearn.feature_extraction.text import CountVectorizer

%matplotlib inline
sns.set(color_codes = True)

In [3]:
df=pd.read_excel('beer-reviews.xlsx')
nltk.download('stopwords')
nltk.download('punkt')
stop_words = set(stopwords.words('english'))

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/rachana/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /Users/rachana/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [6]:
stop = stopwords.words('english')
def remove_stopwords(text):
    words = nltk.word_tokenize(text)
    stop_words = set(stopwords.words("english"))
    filtered_words = [word for word in words if word.lower() not in stop_words]
    filtered_words = [word for word in filtered_words if word.isalnum()]
    return " ".join(filtered_words)
df['body']=df['body'].str.lower()

df['body'] = df['body'].astype(str).apply(remove_stopwords)
df['body']


0                     amazing stout expensive worth price
1           beer meet exceeded unreal hype simply amazing
2                   let sit warm room little unbelievable
3       small pour ebony dark real head intense rich a...
4       vintage 2022 served tap toppling goliath snuli...
                              ...                        
6626    poured bottle snifter home thanks darin share ...
6627    good god man holy shit really good beer blesse...
6628    appearance bubble show lifts head fullsome thr...
6629    pours sure describe color golden yellow brown ...
6630    tropical nose sweet like sunnyd nice acidity a...
Name: body, Length: 6631, dtype: object

# Task B
Assume that a customer, who will be using this recommender system, has specified 3 attributes
in a product. E.g., one website describes multiple attributes of beer (but you should choose attributes
from the actual data)
Perform a word frequency analysis of beer reviews is a better way
to find important attributes in the actual data.

In [9]:
# Calculate word frequencies using CountVectorizer
vectorizer = CountVectorizer()
X = vectorizer.fit_transform(df['body'])

# Calculate the frequency of each word in the reviews
word_frequencies = pd.DataFrame({'word': vectorizer.get_feature_names_out(), 'frequency': np.array(X.sum(axis=0))[0]})

# Get the top 3 most frequent words (attributes)
top_attributes = word_frequencies.nlargest(15, 'frequency')

# Display the top 3 attributes
print("Top 3 Attributes:")
print(top_attributes)

Top 3 Attributes:
              word  frequency
1831          beer       5268
7322          head       3650
14884        taste       3483
8869         light       2891
8887          like       2537
6917          good       2249
9241          malt       2165
14700        sweet       2145
4341          dark       1945
10497          one       1859
1319         aroma       1825
2867   carbonation       1825
6167        finish       1690
10163         nice       1628
10656      overall       1573


In [ ]:
attributes=['light','malt','sweet','dark']

# Task H
Using the top four attributes of beer (from word frequency analysis), calculate the lifts between
these attributes and any 10 beers in your data. Choose one beer, and find the most similar beer (among
the remaining 9) using the lift values. Explain your method.

In [11]:
def initialize(list1, list2):
    attribute_mentions_per_review = {attribute: set() for attribute in list1 + list2}
    co_mentions = {}
    for attr1 in list1 + list2:
        co_mentions[attr1] = {}
        for brand in list1 + list2:
            co_mentions[attr1][brand] = 0
    individual_mentions = {attr: 0 for attr in list1 + list2}
    lift_ratios = {}
    lift_already_calculated = set()
    return co_mentions, attribute_mentions_per_review, individual_mentions, lift_ratios, lift_already_calculated

In [12]:
def calculate_lift(attr1, brand, co_mentions, individual_mentions, total_posts):
    if attr1 == brand:
        return 0  # Lift ratio between the same brand is 0
    # Calculate lift using the formula: lift(attr1, brand) = (P(attr1 and brand) / (P(attr1) * P(brand))) * N
    p_attr1_and_brand = co_mentions[attr1][brand]
    p_attr1 = individual_mentions[attr1]
    p_brand = individual_mentions[brand]
    if p_attr1 == 0 or p_brand == 0:
        return 0  
    else:
        return (p_attr1_and_brand / (p_attr1 * p_brand)) * total_posts

In [47]:
#selected_beer_brands=["Mornin' Delight","Nelson","Stickee Monkee","The Wild One","Corsendonk Christmas Ale"]
attributes=['light', 'malt', 'sweet', 'dark']
selected_beer_brands=["Old Chub","Voodoo Ranger Juicy Haze IPA","Harvest Ale (Limited Edition)","The Angel's Share - Bourbon Barrel-Aged","Weihenstephaner Korbinian","G'Knight","Mirror Pond Pale Ale","Antarctica Pilsen","Dark Lord","Breakfast Stout"]


In [48]:
lift_df = pd.DataFrame(index=attributes, columns=selected_beer_brands)
for brand in selected_beer_brands:
    for attr in attributes:
        # Calculate support and joint support
        df['body'].fillna('', inplace=True)
        support_attr = len(df[df['body'].str.contains(attr)]) / len(df)
        support_brand = len(df[df['item'] == brand]) / len(df)
        joint_support = len(df[(df['body'].str.contains(attr)) & (df['item'] == brand)]) / len(df)
        
        # Calculate lift
        if support_attr * support_brand == 0:
            lift = 0  # Avoid division by zero
        else:
            lift = joint_support / (support_attr * support_brand)

        lift_df.at[attr, brand] = lift

# Convert lift values to numeric (they may be stored as strings)
#df = df.apply(pd.to_numeric, errors='coerce')
# Select ten beer brands (replace with your selection criteria)


In [49]:
lift_df

,Old Chub,Voodoo Ranger Juicy Haze IPA,Harvest Ale (Limited Edition),The Angel's Share - Bourbon Barrel-Aged,Weihenstephaner Korbinian,G'Knight,Mirror Pond Pale Ale,Antarctica Pilsen,Dark Lord,Breakfast Stout
light,1.050063,0.816716,1.429253,0.918805,0.890963,0.918805,1.225074,1.225074,0.700042,0.735044
malt,2.146416,0.0,0.50083,0.375623,2.458622,1.878114,1.502491,1.001661,0.858566,0.300498
sweet,2.692776,0.349064,2.094381,1.178089,1.713585,1.570786,0.523595,0.0,2.24398,1.256629
dark,3.897439,0.0,1.818805,3.410259,3.968301,2.046155,0.0,0.0,4.676927,2.182566


In [51]:
# Choose a reference beer (replace with the beer you want to use as reference)
reference_beer = 'Weihenstephaner Korbinian'

# Create a dictionary to store the most similar beer for each reference attribute
most_similar_beers = {}

# Iterate through the remaining nine beers
for beer in selected_beer_brands:
    if beer != reference_beer:  # Exclude the reference beer itself
        most_similar_attribute = None
        highest_lift = 0.0

        # Find the attribute with the highest lift value when associated with the reference beer
        for attribute in attributes:
            lift_value = lift_df.at[attribute, beer]
            if lift_value > highest_lift:
                highest_lift = lift_value
                most_similar_attribute = attribute

        # Store the most similar beer for the reference beer
        most_similar_beers[beer] = (most_similar_attribute, highest_lift)

# Find the beer with the highest lift value
most_similar_beer = max(most_similar_beers, key=lambda x: most_similar_beers[x][1])
similar_attribute, highest_lift = most_similar_beers[most_similar_beer]

print(f"The most similar beer to {reference_beer} is {most_similar_beer} based on the attribute '{similar_attribute}' with a lift value of {highest_lift}.")

The most similar beer to Weihenstephaner Korbinian is Dark Lord based on the attribute 'dark' with a lift value of 4.676926691729323.
